In [ ]:
from tools import *


2024-05-26 22:45:17,569 ib_async.wrapper INFO Warning 2157, reqId -1: Sec-def data farm connection is broken:secdefil
2024-05-26 22:45:18,254 ib_async.wrapper INFO Warning 2158, reqId -1: Sec-def data farm connection is OK:secdefil
2024-05-26 23:12:57,332 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-05-26 23:12:58,680 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-05-26 23:13:30,364 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-05-26 23:13:31,785 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-05-26 23:19:45,063 ib_async.wrapper INFO Warning 2103, reqId -1: Market data farm connection is broken:usfarm
2024-05-26 23:19:46,171 ib_async.wrapper INFO Warning 2104, reqId -1: Market data farm connection is OK:usfarm
2024-05-26 23:20:26,475 ib_async.wrapper INFO position: Position(account='U10394496', cont

In [28]:
util.df([t.order for t in ib.openTrades()])


,orderId,clientId,permId,action,totalQuantity,orderType,lmtPrice,auxPrice,tif,activeStartTime,...,usePriceMgmtAlgo,duration,postToAts,advancedErrorOverride,manualOrderTime,minTradeQty,minCompeteSize,competeAgainstBestOffset,midOffsetAtWhole,midOffsetAtHalf
0,379,6,1735430611,BUY,1.0,LMT,18670.25,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
1,0,0,2099801404,BUY,1.0,LMT,18040.50,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
2,419,28,2104249594,SELL,1.0,LMT,18863.00,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308
3,416,66,527389863,SELL,1.0,LMT,18925.00,0.0,GTC,,...,False,2147483647,2147483647,,,2147483647,2147483647,1.797693e+308,1.797693e+308,1.797693e+308


In [ ]:
util.df(ib.positions()).sort_values("account", ascending=False)


In [ ]:
trades = ib.trades()
trades[-1]


In [ ]:
trades = parse_ibrecords(ib.trades())
submittedTrades = [o for o in trades if o["status"] == "Submitted"]

print_ibrecords_table(
    submittedTrades,
    ["orderId", "permId", "status", "orderType", "action", "lmtPrice", "remaining"],
)


In [ ]:
# openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

# openTrades = parse_ibrecords(ib.openTrades())
# # util.df([d for d in openTrades])

# print_ibrecords_table(openTrades)

# openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

# trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

# orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


openOrders = parse_ibrecords(ib.openOrders())
openOrders


In [ ]:
print_ibrecords_table(
    openOrders,
    cols=[
        "localSymbol",
        "permId",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
    ],
)


In [ ]:
ord = ib.orders()

[print(o) for o in ord if o.permId == 1394515162]


In [ ]:
util.df(ib.orders())


In [ ]:
util.df(ib.openOrders())


In [ ]:
from datetime import datetime

input_string = "20240930 16:00:00 Central Standard Time"
date_format = "%Y%m%d %H:%M:%S %Z"

try:
    datetime_object = datetime.strptime(input_string, date_format)
    print(datetime_object)
except ValueError:
    print("Incorrect date format.")


In [ ]:
util.df([t for t in ib.reqAllOpenOrders()])


In [ ]:
clear_output()
o = ib.orders()[0]
o = util.dataclassNonDefaults(o)
o.pop("softDollarTier", None)
o.pop("shareholder", None)
o.pop("autoCancelDate", None)
o.pop("auxPrice", None)

data, count = supabase.table("orders").upsert(o).execute()

util.tree(o)


In [ ]:
o["autoCancelDate"]


In [ ]:
od = util.dataclassAsDict(o)
print(od)


In [ ]:
clear_output()
o = ib.orders()[0]
od = util.dataclassAsDict(o)

data, count = supabase.table("orders").upsert(od).execute()


In [ ]:
clear_output()
util.dataclassRepr(ib.trades()[0])

# util.dataclassAsDict(executions[0])


In [ ]:
# Get Account details
details = ib.accountSummary()
details_df = util.df(details)

# Tag Values
# tags = details_df['tag'].values.tolist()
# print(tags)

# Set tag values as index
details_df = details_df.set_index("tag")
net_liquidation_value = details_df.loc["NetLiquidation", "value"]
details_df


In [ ]:
util.tree(ib.trades()[0])


In [ ]:
util.tree(ib.openTrades()[0])


In [ ]:
util.tree(ib.openTrades()[1])


In [ ]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != "NQ":
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


In [32]:
trades_df = util.df(ib.trades())
trades_df


,contract,order,orderStatus,fills,log,advancedError
0,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249593, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
1,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249578, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
2,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249574, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
3,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249575, action='SELL', orderT...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
4,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(orderId=379, clientId=6, permId=17354306...","OrderStatus(orderId=379, status='Submitted', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
5,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2099801404, action='BUY', totalQu...","OrderStatus(orderId=0, status='Submitted', fil...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
6,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(orderId=419, clientId=28, permId=2104249...","OrderStatus(orderId=419, status='Submitted', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",
7,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(orderId=416, clientId=66, permId=5273898...","OrderStatus(orderId=416, status='Submitted', f...",[],"[TradeLogEntry(time=datetime.datetime(2024, 5,...",


In [33]:
# get all the trades where fills is not []
fills = trades_df[trades_df.fills.map(len) > 0]

fills_list = list(fills["order"])

# loop through fills and convert each member of this data type to a row in a df
#  Order(permId=253828901, action='BUY', orderType='LMT', lmtPrice=17580.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U10394496', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20240930 16:00:00 Central Standard Time', filledQuantity=1.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder')


In [35]:
fills


,contract,order,orderStatus,fills,log,advancedError
0,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249593, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
1,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249578, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
2,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249574, action='BUY', orderTy...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
3,"Future(conId=620730920, symbol='NQ', lastTrade...","Order(permId=2104249575, action='SELL', orderT...","OrderStatus(orderId=0, status='Filled', filled...","[(Future(conId=620730920, symbol='NQ', lastTra...","[TradeLogEntry(time=datetime.datetime(2024, 5,...",
